## Obtaining baseline results using Yolo models 

### Data

#### Converting Geojson to YOLO

In [10]:
import os
import numpy as np
from glob import glob
from PIL import Image

def fix_path(path: str) -> str:
    return path.replace("\\", "/")

# Directories
labels_dir = fix_path(r"O:\Capstone_2\Marine-Debris-Detection\NASA_Planet_Data\labels")
source_dir = fix_path(r"O:\Capstone_2\Marine-Debris-Detection\NASA_Planet_Data\source")
output_dir = fix_path("O:/Capstone_2/Marine-Debris-Detection/yolo_labels")

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)


In [ ]:

# Class mapping
class_map = {
    "marine_debris": 0  # Update with additional classes if needed
}

# Normalize bounding box coordinates
def normalize_bbox(bbox, img_width, img_height):
    x_min, y_min, x_max, y_max = bbox
    x_center = (x_min + x_max) / 2 / img_width
    y_center = (y_min + y_max) / 2 / img_height
    width = (x_max - x_min) / img_width
    height = (y_max - y_min) / img_height
    return x_center, y_center, width, height

# Process .npy files
npy_files = glob(os.path.join(labels_dir, "*.npy"))

for npy_file in npy_files:
    # Load .npy data
    data = np.load(npy_file, allow_pickle=True).tolist()

    # Get corresponding image dimensions
    base_name = os.path.splitext(os.path.basename(npy_file))[0]
    image_path = os.path.join(source_dir, base_name + ".jpg")
    
    if not os.path.exists(image_path):
        print(f"Image not found for {base_name}, skipping...")
        continue

    with Image.open(image_path) as img:
        img_width, img_height = img.size

    # Create YOLO label file
    yolo_file_path = os.path.join(output_dir, base_name + ".txt")
    with open(yolo_file_path, "w") as yolo_file:
        for entry in data:
            x_min, y_min, x_max, y_max, class_id = entry
            if class_id > 0:
                class_id -= 1

            # Normalize the bounding box coordinates
            x_center, y_center, width, height = normalize_bbox(
                (x_min, y_min, x_max, y_max), img_width, img_height
            )

            # Write to YOLO file
            yolo_file.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

print("Conversion complete!")


Conversion complete!


### Splitting Dataset

In [1]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Define paths
labels_dir = r"O:/Capstone_2/Marine-Debris-Detection/yolo_labels"
images_dir = r"O:/Capstone_2/Marine-Debris-Detection/NASA_Planet_Data/source"
output_dir = r"O:/Capstone_2/Marine-Debris-Detection/dataset_splits"

# Ratios
train_ratio = 0.7
val_ratio = 0.2
test_ratio = 0.1

# Ensure the ratios sum to 1
assert round(train_ratio + val_ratio + test_ratio, 5) == 1.0, "Ratios must sum to 1."

# Get list of label files
label_files = [f for f in os.listdir(labels_dir) if f.endswith(".txt")]

# Split into train, validation, and test
train_files, temp_files = train_test_split(label_files, test_size=(1 - train_ratio), random_state=42, shuffle=True)
val_files, test_files = train_test_split(temp_files, test_size=(test_ratio / (test_ratio + val_ratio)), random_state=42, shuffle=True)

# Function to move files
def move_files(files, dest_images, dest_labels):
    os.makedirs(dest_images, exist_ok=True)
    os.makedirs(dest_labels, exist_ok=True)
    for label_file in files:
        # Move label file
        src_label = os.path.join(labels_dir, label_file)
        dest_label = os.path.join(dest_labels, label_file)
        shutil.copy(src_label, dest_label)

        # Move corresponding image file
        image_file = label_file.replace(".txt", ".jpg")  # Assuming images are .jpg
        src_image = os.path.join(images_dir, image_file)
        dest_image = os.path.join(dest_images, image_file)
        if os.path.exists(src_image):
            shutil.copy(src_image, dest_image)
        else:
            print(f"Image file {image_file} not found for label {label_file}")

# Move files to respective directories
move_files(train_files, os.path.join(output_dir, "train/images"), os.path.join(output_dir, "train/labels"))
move_files(val_files, os.path.join(output_dir, "val/images"), os.path.join(output_dir, "val/labels"))
move_files(test_files, os.path.join(output_dir, "test/images"), os.path.join(output_dir, "test/labels"))

print("Dataset split completed.")
print(f"Train files: {len(train_files)}")
print(f"Validation files: {len(val_files)}")
print(f"Test files: {len(test_files)}")


Dataset split completed.
Train files: 517
Validation files: 148
Test files: 74
